In [1]:
%matplotlib inline
import os 
import pandas as pd
import matplotlib.image as mpimg
# import shutil
import matplotlib.pyplot as plt
import pickle
import glob
import numpy as np
from PIL import Image
import tensorflow as tf

In [2]:
photoangularity = []
phototexture    = []

folder=os.getcwd()+'/photos/'+'texture/'
texture_dict = {}

filelist = glob.glob(folder+'/*.png')
x = np.array([np.array(Image.open(fname)).ravel() for fname in filelist])

In [3]:
keys = []
for files in filelist:
    keys.append(files.split('/')[-1].replace('.png',''))

In [4]:
texture_dict = dict(zip(keys, zip(x)))

In [5]:
tf.image.decode_png(filelist[0],channels=3)

<tf.Tensor 'DecodePng:0' shape=(?, ?, 3) dtype=uint8>

In [19]:
x[0]

array([168, 168, 159, ..., 140, 145, 139], dtype=uint8)